In [ ]:
import sys
import os
import json
import time

In [ ]:
# include path to our library
cwd = os.getcwd()
dir_path = os.path.dirname(os.path.realpath(cwd))
sys.path.append(dir_path)
time.sleep(2)

In [ ]:
dir_path
%cd $dir_path
!ls

In [ ]:
import pandas as pd
from core.utils.utils import *

## Reading the data ##

In [ ]:
#configs = json.load(open('config_test_cnn.json', 'r'))
#configs = json.load(open('config-mlp.json', 'r'))
#configs = json.load(open('config_cnn.json', 'r'))
configs = json.load(open('config_lstm.json', 'r'))


output_path = configs['paths']['save_dir']
cylindrical = configs['data']['cylindrical']  # set to polar or cartesian coordenates
normalise = configs['data']['normalise'] 
name = configs['model']['name']
type_pred = configs['testing']['type_prediction']
data_file = configs['data']['filename']

#create a encryp name for dataset
path_to, filename = os.path.split(data_file)

orig_ds_name = filename

encryp_ds_name = get_unique_name(orig_ds_name)
decryp_ds_name = get_decryp_name(encryp_ds_name)

output_encry = os.path.join(output_path, encryp_ds_name)  

if cylindrical:
    coord = 'cylin'
else:
    coord = 'xyz'

print('source file for testing:\n\t ', data_file)
print('predicted points: ')
path1 = os.path.join(output_encry, 'x_true_%s_%s_%s.csv' % (name, coord, type_pred))
path2 = os.path.join(output_encry, 'y_true_%s_%s_%s.csv' % (name, coord, type_pred))
path3 = os.path.join(output_encry, 'y_pred_%s_%s_%s.csv' % (name, coord, type_pred))

print('\tloading from .. %s' % path1)
print('\tloading from .. %s' % path2)
print('\tloading from .. %s' % path3)

df_test = pd.read_csv(path1, header=None)
df_true = pd.read_csv(path2, header=None)
df_pred = pd.read_csv(path3, header=None)

print('shape df_test ', df_test.shape)
print('shape df_true ', df_true.shape)
print('shape df_pred ', df_pred.shape)
# concat
#y_true = pd.concat([df_test, df_true], axis = 1, ignore_index = True)
#y_pred = pd.concat([df_test, df_pred], axis = 1, ignore_index = True)

y_true = np.concatenate([df_test, df_true], axis = 1)
y_pred = np.concatenate([df_test, df_pred], axis = 1)
y_true = pd.DataFrame(y_true)
y_pred = pd.DataFrame(y_pred)
y_true.name = 'real'
y_pred.name = 'pred'

print('size y_true ', y_true.shape)
print('size y_pred ', y_pred.shape)


## Ploting Tracks Predictions ##

In [ ]:
# Dados originais
N_tracks = 30
path_html = ''
output_path = 'results'
if cylindrical:
    path_html=os.path.join(output_encry, 'track-original_cylin.html')
else:
    path_html=os.path.join(output_encry, 'track-original_xyz.html')
    
fig = track_plot_xyz([y_true], n_hits = 10, cylindrical = cylindrical, n_tracks = N_tracks, 
               path=path_html, title='Track Originals - Coordenates %s' % coord.upper() , auto_open=True)

fig.show()


In [ ]:
# predicted data
path_html = ''
N_tracks = 30
output_path = 'results'

if cylindrical:
    path_html=os.path.join(output_encry, 'track-predicted-%s_cylin_%s.html' % (name, type_pred))
else:
    path_html=os.path.join(output_encry, 'track-predicted-%s_xyz_%s.html' % (name, type_pred))
    
fig = track_plot_xyz([y_true, y_pred], n_hits = 10, cylindrical = cylindrical, n_tracks = N_tracks, 
               path=path_html, title='Track Predictions - Coordenates %s (%s-%s)' % 
                     (coord, name.upper(), type_pred.upper()), auto_open=True)

fig.show()